## <font color='cyan'> Testing on user input image <font/>

## <font color='gold'> ANN Testing<font/>

In [6]:
import torch
from torchvision import transforms
from PIL import Image
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 10),
        )

    def forward(self, features):
        out = self.network(features)
        return out

model = Model(num_features=784)
model.load_state_dict(torch.load("./model/(ANN)digit_recognition_model.pth",weights_only=True))
model.eval()

Model(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [2]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),  # Resize to 28x28 pixels
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [23]:
def predict_digit_by_ANN(image_path):
    # Open the image
    image = Image.open(image_path)

    # Apply the transformation
    image = transform(image).unsqueeze(0)  # Add batch dimension (shape: 1, 1, 28, 28)

    # Flatten the image to (1, 784)
    image = image.view(-1, 28*28)  # Flatten the image

    # Perform the inference
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        predicted_digit = predicted.item()

    return predicted_digit


In [33]:
image_paths = ["../digit2S.png", "../digit3.png","../digit3a.png","../digit4.png","../digit5.png", "../digit6.png","../digit6a.png", "../digit7.png","../digit7S.jpg", "../digit8.png", "../digit9.png","../digit9a.png"]

for image_path in image_paths:
    predicted_digit = predict_digit_by_ANN(image_path)
    print(f"Predicted digit for {image_path}: {predicted_digit}")

Predicted digit for ../digit2S.png: 2
Predicted digit for ../digit3.png: 2
Predicted digit for ../digit3a.png: 0
Predicted digit for ../digit4.png: 0
Predicted digit for ../digit5.png: 6
Predicted digit for ../digit6.png: 6
Predicted digit for ../digit6a.png: 6
Predicted digit for ../digit7.png: 6
Predicted digit for ../digit7S.jpg: 2
Predicted digit for ../digit8.png: 2
Predicted digit for ../digit9.png: 0
Predicted digit for ../digit9a.png: 2


## <font color='gold'> CNN Testing<font/>

In [4]:
import torch.nn.functional as F

In [7]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Matches the training architecture
        self.fc2 = nn.Linear(128, 10)

    def forward(self, feature):
        feature = F.relu(self.conv1(feature))
        feature = self.pool(feature)
        feature = F.relu(self.conv2(feature))
        feature = self.pool(feature)
        feature = feature.view(feature.size(0), -1)  # Flatten
        feature = F.relu(self.fc1(feature))
        feature = self.fc2(feature)
        return feature

cnn_model = CNNModel()

# Load the saved state dictionary
cnn_model.load_state_dict(torch.load('./model/(CNN)digit_recognition_model.pth', map_location=torch.device('cpu'),weights_only=True))

cnn_model.eval()


CNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [8]:
from app.preprocessing import preprocess_image

In [9]:
def predict_digit_by_CNN(image_path):
    # Preprocess the image
    image_tensor = preprocess_image(image_path)

    # Perform inference
    with torch.no_grad():
        outputs = cnn_model(image_tensor)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Get predicted class
        predicted_digit = predicted.item()

    return predicted_digit

In [10]:
image_paths = ["../digit2S.png","../digit3a.png","../digit4.png","../digit5.png","../digit6a.png", "../digit7.png","../digit7S.jpg", "../digit8.png","../digit9a.png"]

for image_path in image_paths:
    predicted_digit = predict_digit_by_CNN(image_path)
    print(f"Predicted digit for {image_path}: {predicted_digit}")


Predicted digit for ../digit2S.png: 2
Predicted digit for ../digit3a.png: 3
Predicted digit for ../digit4.png: 4
Predicted digit for ../digit5.png: 5
Predicted digit for ../digit6a.png: 6
Predicted digit for ../digit7.png: 7
Predicted digit for ../digit7S.jpg: 7
Predicted digit for ../digit8.png: 8
Predicted digit for ../digit9a.png: 9


***